# DTW

In [19]:
import sys
from math import exp, log
import numpy as np

rng = np.random.RandomState(0)
X = rng.randn(5, 4)
Z = rng.randn(6, 4)

def dtw(x, y, dist):
    """
    Computes Dynamic Time Warping (DTW) of two sequences.
    :param array x: N1*M array
    :param array y: N2*M array
    :param func dist: distance used as cost measure
    Returns the minimum distance, the cost matrix, the accumulated cost matrix, and the wrap path.
    """
    assert len(x)
    assert len(y)
    r, c = len(x), len(y)
    D0 = np.zeros((r + 1, c + 1))
    D0[0, 1:] = np.inf
    D0[1:, 0] = np.inf
    D1 = D0[1:, 1:] # view
    
    for i in range(r):
        for j in range(c):
            D1[i, j] = dist(x[i].reshape(1,-1), y[j].reshape(1,-1))
    C = D1.copy()
    for i in range(r):
        for j in range(c):
            D1[i, j] += min(D0[i, j], D0[i, j+1], D0[i+1, j])
    return D1[-1, -1] , C, D1


dist=lambda a,b: np.linalg.norm(a-b)
values=dtw(X,Z,dist=dist)
distance=values[0]
D=values[2]
print('distance',distance)
#D

('distance', 17.40784575188877)


# Soft DTW

In [20]:
import sys
from math import exp, log
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

def _softmin3( a, b, c,gamma):
    a /= -gamma
    b /= -gamma
    c /= -gamma
    max_val = max(max(a, b), c)
    tmp = 0
    tmp += exp(a - max_val)
    tmp += exp(b - max_val)
    tmp += exp(c - max_val)
    return -gamma * (log(tmp) + max_val)

def _soft_dtw(D,R,gamma):
    m = D.shape[0]
    n = D.shape[1]
    for i in range(m + 1):
        R[i, 0] = np.inf
    for j in range(n + 1):
        R[0, j] = np.inf
        
    R[0, 0] = 0
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            R[i, j] = D[i-1, j-1] + _softmin3(R[i-1, j], R[i-1, j-1],R[i, j-1],gamma)

def SoftDTW(D, gamma=1.0):
    m, n = D.shape
    R_ = np.zeros((m+2, n+2), dtype=np.float64)
    _soft_dtw(D, R_, gamma=gamma)
    return R_[m, n],R_


D = euclidean_distances(X, Z)
value = SoftDTW(D, gamma=0.01)
print(value[0])

17.407842802833436
